In [2]:
# STEP 1: Upload file (must be its own cell)
import platform, torch

In [3]:
%pip -q install "transformers==4.46.3" "tokenizers==0.20.3" einops addict easydict pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 71.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 36.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ONE-CELL: pick latest uploaded image automatically → run DeepSeek-OCR


import os, glob, time, torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

assert torch.cuda.is_available(), "GPU is OFF. Runtime → Change runtime type → GPU, then rerun."

# 1) Find most recent image in /content if FNAME is missing
def pick_latest_image():
    pats = ["/content/*.png", "/content/*.jpg", "/content/*.jpeg", "/content/*.webp"]
    files = []
    for p in pats:
        files += glob.glob(p)
    assert files, "No image file found in /content. If you just uploaded, it’s saved in /content; try again."
    files.sort(key=lambda f: os.path.getmtime(f), reverse=True)
    return files[0]

img_path = "/kaggle/input/test-img/test.png"

if not img_path:
    img_path = pick_latest_image()

print("Using image:", img_path)

# 2) Optional: shrink giant screenshots for speed
img = Image.open(img_path).convert("RGB")
if max(img.size) > 2000:
    s = 2000 / max(img.size)
    img = img.resize((int(img.width*s), int(img.height*s)))
    img_path_proc = "/content/_shrunk.png"
    img.save(img_path_proc, optimize=True)
else:
    img_path_proc = img_path


Using image: /kaggle/input/test-img/test.png


In [4]:

# 3) Load model (needs eager attention)
model_id = 'deepseek-ai/DeepSeek-OCR'
tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
if tok.pad_token is None and tok.eos_token is not None:
    tok.pad_token = tok.eos_token

t0 = time.time()
model = AutoModel.from_pretrained(
    model_id,
    trust_remote_code=True,
    use_safetensors=True,
    attn_implementation="eager"   # required for this arch
).to(dtype=torch.bfloat16, device="cuda").eval()
print(f"Model loaded in {time.time()-t0:.1f}s")
# model = model.eval().cuda().to(torch.bfloat16)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

modeling_deepseekocr.py: 0.00B [00:00, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-OCR:
- deepencoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/deepseek-ai/DeepSeek-OCR:
- modeling_deepseekocr.py
- conversation.py
- modeling_deepseekv2.py
- deepencoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
2025-11-08 19:11:43.547047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762629103.764750      39 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-000001.safetensors:   0%|          | 0.00/6.67G [00:00<?, ?B/s]

Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at deepseek-ai/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded in 89.3s


In [5]:



# 4) Fast OCR for screenshots (no tiling)
prompt = "<image>\nFree OCR."
outdir = "/kaggle/working/"; os.makedirs(outdir, exist_ok=True)

@torch.inference_mode()
def run_ocr(path):
    t = time.time()
    res = model.infer(
        tok,
        prompt=prompt,
        image_file=path,
        output_path=outdir,
        base_size=768,       # smaller canvas = faster
        image_size=512,      # smaller tiles
        crop_mode=False,     # screenshots usually don't need tiling
        save_results=True,
        test_compress=False
    )
    print(f"[OK] {os.path.basename(path)} in {time.time()-t:.1f}s")
    return res

out = run_ocr(img_path_proc)
print("Saved files in:", outdir)
print(out)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


directly resize


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.


BASE:  torch.Size([1, 64, 1280])
NO PATCHES
# Welcome to Borrowwell

We're having trouble finding your credit score...

We couldn't match the information you provided with a profile in Equifax Canada's records.

## What you can do:

1. **Check your Borrowwell user profile**  
   Make sure all information (name, date of birth, and address) is correct and up-to-date. If you need assistance in making these changes please reach out to us at [info@borwell.com](mailto:info@borwell.com).

2. **Contact Equifax Canada**  
   Confirm if your Borrowwell profile information matches your Equifax Canada file and for any active alerts. If you are unsure about what information Equifax Canada has on you, please contact them directly at 1-866-829-5961 to verify.

3. **Contact Borrowwell**  
   Once all information is correct and matches your Equifax Canada file, please contact us at [info@borwell.com](mailto:info@borwell.com) to have your identity verification questions reset.
===============save result

image: 0it [00:00, ?it/s]
other: 0it [00:00, ?it/s]

[OK] _shrunk.png in 8.8s
Saved files in: /kaggle/working/
None


In [6]:
import ipywidgets as widgets
from IPython.display import display

text_input = widgets.Text(value='', description='Enter text:')
display(text_input)

def on_button_click(b):
    print(f"Input submitted: {text_input.value}")

button = widgets.Button(description="Submit")
button.on_click(on_button_click)
display(button)


Text(value='', description='Enter text:')

Button(description='Submit', style=ButtonStyle())

In [8]:
# Load DeepSeek OCR model and tokenizer
model_name = "unsloth/DeepSeek-OCR"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def ocr_with_deepseek(image):
    if image is None:
        return "Please upload an image."
    image.save("temp.jpg")  # Save temporarily for model processing
    prompt = "<image>\nFree OCR."
    result = model.infer(
        tokenizer,
        prompt=prompt,
        image_file="temp.jpg",
        base_size=1024,
        image_size=640,
        crop_mode=True,
        save_results=False,
    )
    return result['text']

# Gradio UI Setup
iface = gr.Interface(
    fn=ocr_with_deepseek,
    inputs=gr.Image(type="pil"),
    outputs=gr.Textbox(label="OCR Output"),
    title="DeepSeek OCR Web App",
    description="Upload an image and get text extracted by DeepSeek OCR"
)

iface.launch()

The repository for unsloth/DeepSeek-OCR contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/unsloth/DeepSeek-OCR.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  N


ValueError: Loading unsloth/DeepSeek-OCR requires you to execute the configuration file in that repo on your local machine. Make sure you have read the code there to avoid malicious use, then set the option `trust_remote_code=True` to remove this error.

In [1]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()   

* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://151b6c146fff993fc2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
import gradio as gr
from transformers import AutoTokenizer, AutoModel
import torch, time, os

model_id = 'deepseek-ai/DeepSeek-OCR'
tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
if tok.pad_token is None and tok.eos_token is not None:
    tok.pad_token = tok.eos_token

model = AutoModel.from_pretrained(model_id, trust_remote_code=True,
                                  use_safetensors=True,
                                  attn_implementation="eager"
                                 ).to(dtype=torch.bfloat16, device="cuda").eval()

prompt = "<image>\nFree OCR."


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at deepseek-ai/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


* Running on local URL:  http://127.0.0.1:7862
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://d68792b1e6f7e5d851.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


directly resize


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
The attention layers in this model are transitioning from computing the RoPE embeddings internally through `position_ids` (2D tensor with the indexes of the tokens), to using externally computed `position_embeddings` (Tuple of tensors, containing cos and sin). In v4.46 `position_ids` will be removed and `position_embeddings` will be mandatory.


BASE:  torch.Size([1, 64, 1280])
NO PATCHES
# Welcome to Borrowwell

We're having trouble finding your credit score...

We couldn't match the information you provided with a profile in Equifax Canada's records.

## What you can do:

1. **Check your Borrowwell user profile**  
   Make sure all information (name, date of birth, and address) is correct and up-to-date. If you need assistance in making these changes please reach out to us at [info@borwell.com](mailto:info@borwell.com).

2. **Contact Equifax Canada**  
   Confirm if your Borrowwell profile information matches your Equifax Canada file and for any active alerts. If you are unsure about what information Equifax Canada has on you, please contact them directly at 1-866-829-5961 to verify.

3. **Contact Borrowwell**  
   Once all information is correct and matches your Equifax Canada file, please contact us at [info@borwell.com](mailto:info@borwell.com) to have your identity verification questions reset.
===============save result

image: 0it [00:00, ?it/s]
other: 0it [00:00, ?it/s]


In [11]:
import numpy as np
from PIL import Image

In [17]:
outdir = "/kaggle/working/test"; 
# os.makedirs(outdir, exist_ok=True)
# outdir = "./results"
os.makedirs(outdir, exist_ok=True)

In [23]:
@torch.inference_mode()
def run_ocr(image):
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)
    # image_path = "/kaggle/input/test-img/test.png" 
    # Define Kaggle input directory
    input_dir = "/kaggle/working/"
    os.makedirs(input_dir, exist_ok=True)  # Ensure directory exists

    # Generate a unique filename (to avoid overwriting files)
    image_filename = f"test_21.png"
    image_path = os.path.join(input_dir, image_filename)

    # Save uploaded image to the Kaggle input directory
    image.save(image_path)
    # image.save(image_path)
    res = model.infer(tok, prompt=prompt, image_file=image_path,
                      output_path=outdir, base_size=768,
                      image_size=512, crop_mode=False, save_results=True)
    # print(type(res))
    # Path to results.mmd
    text_file = os.path.join(outdir, "results.mmd")
    
    if os.path.exists("/kaggle/working/test/result.mmd"):
        with open("/kaggle/working/test/result.mmd", "r") as f:
            ocr_text = f.read()
    else:
        ocr_text = "OCR completed, but text file not found."
    return ocr_text




ui = gr.Interface(fn=run_ocr, inputs="image", outputs="text",
                  title="DeepSeek OCR", description="Upload an image to extract text.")
ui.launch()

* Running on local URL:  http://127.0.0.1:7874
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://558954394df0529bbf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


directly resize
BASE:  torch.Size([1, 64, 1280])
NO PATCHES
# Welcome to Borrowwell

We're having trouble finding your credit score...

We couldn't match the information you provided with a profile in Equifax Canada's records.

## What you can do:

1. **Check your Borrowwell user profile**  
   Make sure all information (name, date of birth, and address) is correct and up-to-date. If you need assistance in making these changes please reach out to us at [info@borwell.com](mailto:info@borwell.com).

2. **Contact Equifax Canada**  
   Confirm if your Borrowwell profile information matches your Equifax Canada file and for any active alerts. If you are unsure about what information Equifax Canada has on you, please contact them directly at 1-866-829-5961 to verify.

3. **Contact Borrowwell**  
   Once all information is correct and matches your Equifax Canada file, please contact us at [info@borwell.com](mailto:info@borwell.com) to have your identity verification questions reset.
==========

image: 0it [00:00, ?it/s]
other: 0it [00:00, ?it/s]
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


directly resize
BASE:  torch.Size([1, 64, 1280])
NO PATCHES
# We have experienced an issue processing your request

We're sorry, we are unable to process your request at this time. Please contact our Customer Care Team at 1-866-828-9361 for additional help completing your request.

The Customer Care Team is available between 9:00 AM and 9:00 PM ET, Mon-Fri; 9:00 AM and 6:00 PM ET, Sat-Sun
===============save results:===============


...

directly resize
BASE:  torch.Size([1, 64, 1280])
NO PATCHES
# Welcome to Borrowwell

We're having trouble finding your credit score...

We couldn't match the information you provided with a profile in Equifax Canada's records.

## What you can do:

1. **Check your Borrowwell user profile**  
   Make sure all information (name, date of birth, and address) is correct and up-to-date. If you need assistance in making these changes please reach out to us at [info@borwell.com](mailto:info@borwell.com).

2. **Contact Equifax Canada**  
   Confirm if your Borrowwell profile information matches your Equifax Canada file and for any active alerts. If you are unsure about what information Equifax Canada has on you, please contact them directly at 1-866-829-5961 to verify.

3. **Contact Borrowwell**  
   Once all information is correct and matches your Equifax Canada file, please contact us at [info@borwell.com](mailto:info@borwell.com) to have your identity verification questions reset.
==========

image: 0it [00:00, ?it/s]
other: 0it [00:00, ?it/s]
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


directly resize
BASE:  torch.Size([1, 64, 1280])
NO PATCHES
# We have experienced an issue processing your request

We're sorry, we are unable to process your request at this time. Please contact our Customer Care Team at 1-866-828-9361 for additional help completing your request.

The Customer Care Team is available between 9:00 AM and 9:00 PM ET, Mon-Fri; 9:00 AM and 6:00 PM ET, Sat-Sun
===============save results:===============


image: 0it [00:00, ?it/s]
other: 0it [00:00, ?it/s]
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


directly resize
BASE:  torch.Size([1, 64, 1280])
NO PATCHES
# Annual Statement of Account

**BRIMSBANKAN JIJIYA**  
**BANK OF BANGLADESH**  
**WOMEN'S AND BOY'S TURF**  
**Permanent Number:**  
**SERIES NO.:**  
**SERIAL NO.:**  
**DATE OF ISSUE:**  
**DATE OF END OF YEAR:**  
**DISTRIBUTION:**  
**DISTRIBUTION DATE:**  
**DISTRIBUTION PERIOD:**  
**DISTRIBUTION PERIOD DATE:**  
**DISTRIBUTION PERIOD DATE:**  
**DISTRIBUTION PERIOD:**  
**DISTRIBUTION PERIOD:**  
**DISTRIBUTION PERIOD DATE:**

---

## The In a Readinessment year.

The income earned in the year is not used for the next year.  
The income earned in the year is used for the next year.  
MIT Service Centre under you want to make a change.

---

## Annual payment summary (for page 2 of detail)

| Amount | $0.00 |
|---|---|
| Annual total | $0.00 |

---

## Before November 24, 2025, the following payment is due:

| Category | Amount |
|---|---|
| Due    | $0.00  |

---

**Please be assured that we are after November 24, 2025

In [22]:
with open("/kaggle/working/test/result.mmd", "r") as f:
        result_text = f.read()
print(result_text)

# Welcome to Borrowwell

We're having trouble finding your credit score...

We couldn't match the information you provided with a profile in Equifax Canada's records.

## What you can do:

1. **Check your Borrowwell user profile**  
   Make sure all information (name, date of birth, and address) is correct and up-to-date. If you need assistance in making these changes please reach out to us at [info@borwell.com](mailto:info@borwell.com).

2. **Contact Equifax Canada**  
   Confirm if your Borrowwell profile information matches your Equifax Canada file and for any active alerts. If you are unsure about what information Equifax Canada has on you, please contact them directly at 1-866-829-5961 to verify.

3. **Contact Borrowwell**  
   Once all information is correct and matches your Equifax Canada file, please contact us at [info@borwell.com](mailto:info@borwell.com) to have your identity verification questions reset.
